In [2]:
from configuration.invest_config import invest
import numpy as np
import pandas as pd
import datetime
import os
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display
import matplotlib.pyplot as plt
from plotly.offline import iplot
from varname import nameof

In [3]:
#Leemos cuales son los índices actuales del IBEX35 y sacamos sus códigos
ibex35=pd.read_csv("configuration/codigos_ibex35.csv")
codigos_ibex35=ibex35["codigo"]

In [4]:
#Creación de un CSV para cada valor del ibex35 entre 2018-01-01 y 2023-04-17 con frecuencia semanal
#Guardados en data_ibex35
for codigo in codigos_ibex35:
    data=invest(codigo,"2018-01-01","2023-04-17","1wk")
    df_data=data.resultados()
    ruta="data_ibex35/"+codigo+".csv"
    df_data.to_csv(ruta)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [5]:
#Creamos el codigo para extraer todos los datos de los CSV y asignarlos al nombre de su índice en bolsa
directory="data_ibex35"

for document in os.scandir(directory):
    if document.is_file() and document.name.endswith(".csv"):
        name=document.name.replace(".MC.csv","")
        code=name+"= pd.read_csv("+"\'"+document.path+"\'"+")"
        print(code.replace("\\","/"))
        

ACS= pd.read_csv('data_ibex35/ACS.MC.csv')
ACX= pd.read_csv('data_ibex35/ACX.MC.csv')
AENA= pd.read_csv('data_ibex35/AENA.MC.csv')
AMS= pd.read_csv('data_ibex35/AMS.MC.csv')
ANA= pd.read_csv('data_ibex35/ANA.MC.csv')
ANE= pd.read_csv('data_ibex35/ANE.MC.csv')
BBVA= pd.read_csv('data_ibex35/BBVA.MC.csv')
BKT= pd.read_csv('data_ibex35/BKT.MC.csv')
CABK= pd.read_csv('data_ibex35/CABK.MC.csv')
CLNX= pd.read_csv('data_ibex35/CLNX.MC.csv')
COL= pd.read_csv('data_ibex35/COL.MC.csv')
ELE= pd.read_csv('data_ibex35/ELE.MC.csv')
ENG= pd.read_csv('data_ibex35/ENG.MC.csv')
FDR= pd.read_csv('data_ibex35/FDR.MC.csv')
FER= pd.read_csv('data_ibex35/FER.MC.csv')
GRF= pd.read_csv('data_ibex35/GRF.MC.csv')
IAG= pd.read_csv('data_ibex35/IAG.MC.csv')
IBE= pd.read_csv('data_ibex35/IBE.MC.csv')
ITX= pd.read_csv('data_ibex35/ITX.MC.csv')
LOG= pd.read_csv('data_ibex35/LOG.MC.csv')
MAP= pd.read_csv('data_ibex35/MAP.MC.csv')
MEL= pd.read_csv('data_ibex35/MEL.MC.csv')
MTS= pd.read_csv('data_ibex35/MTS.MC.csv')
NTG

In [6]:
#Asignamos CSV a los índices del IBEX35
ACS= pd.read_csv('data_ibex35/ACS.MC.csv')
ACX= pd.read_csv('data_ibex35/ACX.MC.csv')
AENA= pd.read_csv('data_ibex35/AENA.MC.csv')
AMS= pd.read_csv('data_ibex35/AMS.MC.csv')
ANA= pd.read_csv('data_ibex35/ANA.MC.csv')
ANE= pd.read_csv('data_ibex35/ANE.MC.csv')
BBVA= pd.read_csv('data_ibex35/BBVA.MC.csv')
BKT= pd.read_csv('data_ibex35/BKT.MC.csv')
CABK= pd.read_csv('data_ibex35/CABK.MC.csv')
CLNX= pd.read_csv('data_ibex35/CLNX.MC.csv')
COL= pd.read_csv('data_ibex35/COL.MC.csv')
ELE= pd.read_csv('data_ibex35/ELE.MC.csv')
ENG= pd.read_csv('data_ibex35/ENG.MC.csv')
FDR= pd.read_csv('data_ibex35/FDR.MC.csv')
FER= pd.read_csv('data_ibex35/FER.MC.csv')
GRF= pd.read_csv('data_ibex35/GRF.MC.csv')
IAG= pd.read_csv('data_ibex35/IAG.MC.csv')
IBE= pd.read_csv('data_ibex35/IBE.MC.csv')
ITX= pd.read_csv('data_ibex35/ITX.MC.csv')
LOG= pd.read_csv('data_ibex35/LOG.MC.csv')
MAP= pd.read_csv('data_ibex35/MAP.MC.csv')
MEL= pd.read_csv('data_ibex35/MEL.MC.csv')
MTS= pd.read_csv('data_ibex35/MTS.MC.csv')
NTGY= pd.read_csv('data_ibex35/NTGY.MC.csv')
RED= pd.read_csv('data_ibex35/RED.MC.csv')
SAB= pd.read_csv('data_ibex35/SAB.MC.csv')
SAN= pd.read_csv('data_ibex35/SAN.MC.csv')
TEF= pd.read_csv('data_ibex35/TEF.MC.csv')
UNI= pd.read_csv('data_ibex35/UNI.MC.csv')

In [63]:
valores=[ACS,ACX,AENA,AMS,ANA,ANE,BBVA,BKT,CABK,CLNX,COL,
        ELE,ENG,FDR,FER,GRF,IAG,IBE,ITX,
        LOG,MAP,MEL,MTS,NTGY,RED,SAB,SAN,TEF,UNI]
valores_name=["ACS","ACX","AENA","AMS","ANA","ANE","BBVA","BKT","CABK","CLNX","COL",
        "ELE","ENG","FDR","FER","GRF","IAG","IBE","ITX",
        "LOG","MAP","MEL","MTS","NTGY","RED","SAB","SAN","TEF","UNI"]
dic_valores = {nombre: valor for nombre, valor in zip(valores_name, valores)}


In [9]:
dict_open={}
n=1
for i in valores:
    dict_open[n]=i["Open"]
    n+=1

In [10]:
df_media_total=pd.DataFrame(dict_open).mean(axis=1)
df_media_IBEX35=pd.concat([BBVA["Date"], df_media_total], axis=1)
df_media_IBEX35=df_media_IBEX35.rename(columns={0:"Open"})
#DF de la media de todos los valores (HAY QUE AÑADIR LAS FECHAS Y CREAR EL GRÁFICO)

In [11]:
df_media_IBEX35

,Date,Open
0,2018-01-01,23.139334
1,2018-01-08,24.167598
2,2018-01-15,24.002538
3,2018-01-22,24.192933
4,2018-01-29,24.009568
...,...,...
271,2023-03-13,26.015214
272,2023-03-20,25.014225
273,2023-03-27,25.324428
274,2023-04-03,26.316471


In [12]:

# MEDIA DEL PRECIO DE APERTURA DEL IBEX35
trace1 = go.Scatter(
                    x = df_media_IBEX35['Date'],
                    y = df_media_IBEX35['Open'],
                    name = 'citations',
                    mode= 'lines',
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                    # text = df['university_name'])
                    )

data = [trace1]

layout = dict(title = dict(text='MEDIA PRECIO DE APERTURA IBEX 35', x=0.5),
             xaxis= dict(title= 'FECHA',ticklen= 5),
             yaxis= dict(title= 'MEDIA PRECIO DE APERTURA',ticklen= 5)
           )

fig = go.Figure(data = data, layout=layout)

iplot(fig)

In [82]:
dato=BKT

layout = dict(
    title = dict(text=nameof(BKT), x=0.5),
    xaxis=dict(
        title='Fecha'
    ),
    yaxis=dict(
        title='Valor'
    )
)
fig = go.Figure(data=[go.Candlestick(x=dato['Date'],
                open=dato['Open'],
                high=dato['High'],
                low=dato['Low'],
                close=dato['Close'])],layout=layout
                )

display(fig)

In [64]:
#Intento crear una funcion que devuelva el grafico de cada valor
def grafico_valor(valor_input):

    layout = dict(
        title = dict(text=valor_input, x=0.5),
        xaxis=dict(
            title='Fecha'
        ),
        yaxis=dict(
            title='Valor'
        )
    )
    fig = go.Figure(data=[go.Candlestick(x=dic_valores[valor_input]['Date'],
                    open=dic_valores[valor_input]['Open'],
                    high=dic_valores[valor_input]['High'],
                    low=dic_valores[valor_input]['Low'],
                    close=dic_valores[valor_input]['Close'])],layout=layout
                    )
    return(fig)
    # display(fig)

In [66]:
grafico_valor("AENA")

In [29]:
max_open = BKT["Open"].max()
max_open_rows = BKT[BKT["Open"] == max_open]
max_open_data = max_open_rows.loc[:, :]

max_open_data

,Date,Open,High,Low,Close,Adj Close,Volume
4,2018-01-29,9.222,9.416,9.148,9.172,7.303559,12573595


In [33]:
for valor in valores:
datos_2020=nameof(valor)[AENA["Date"].str.startswith("2020")]
max_open=AENA[AENA["Open"]==AENA_2020["Open"].max()]
print(valor)
print(max_open)

,Date,Open,High,Low,Close,Adj Close,Volume
107,2020-01-20,172.800003,174.350006,166.600006,168.5,157.374298,611877
